In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

In [ ]:
#Load the data
df = pd.read_csv('cars_train.csv', sep='\t', header=0, encoding="utf-16")

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df

In [ ]:
booleans = ['blindado','entrega_delivery','troca','elegivel_revisao','dono_aceita_troca','veiculo_único_dono','revisoes_concessionaria','ipva_pago','veiculo_licenciado','garantia_de_fábrica','revisoes_dentro_agenda']
numericas = ['num_fotos','ano_de_fabricacao','ano_modelo','hodometro','num_portas','veiculo_alienado','preco']
strings = ['marca','modelo','versao','cambio','tipo','cor','tipo_vendedor','cidade_vendedor','estado_vendedor','anunciante']

In [ ]:
for col in booleans:

    coluna = df[col].value_counts(dropna=False)
    ax = coluna.plot(kind="bar")
    plt.title('Value counts of '+col+' column')
    plt.xlabel(col)
    plt.xticks(rotation=0)
    plt.ylabel('Count')
    for i, v in enumerate(coluna):
        ax.text(i, v, str(v), ha='center', va='bottom', fontsize=12)
    plt.show()

In [ ]:
#Obtendo informação do tipo de combustivel
versao = df['versao']

df['combustivel'] = None

flex=0
gasolina=0
diesel = 0
hibrido = 0
eletrico = 0
outros = 0

for i,ver in enumerate(versao):
    if "FLEX" in ver:
        flex+=1
        df.at[i, 'combustivel'] = 'FLEX'
        continue
    if "GASOLINA" in ver:
        gasolina+=1
        df.at[i, 'combustivel'] = 'GASOLINA'
        continue
    if "DIESEL" in ver:
        diesel+=1
        df.at[i, 'combustivel'] = 'DIESEL'
        continue
    if "HÍBRIDO" in ver or "HYBRID" in ver:
        hibrido+=1
        df.at[i, 'combustivel'] = 'HIBRIDO'
        continue
    if "ELÉTRICO" in ver:
        eletrico+=1
        df.at[i, 'combustivel'] = 'ELETRICO'
        continue
    print(ver)
    df.at[i, 'combustivel'] = 'OUTROS'
    outros+=1

print(flex)
print(gasolina)
print(diesel)
print(hibrido)
print(eletrico)
print(outros)


In [ ]:
# sns.pairplot(df)

In [ ]:
tipo = df['tipo_vendedor']
anunciante = df['anunciante']

ok = 0


for t, a in zip(tipo, anunciante):
    if t=='PF' and a=='Pessoa Física':
#         ok+=1
        continue
    if (t=='PJ' and a=='Loja') or(t=='PJ' and a=='Concessionária'):
#         ok+=1
        continue
    print("Tipo:", t, "- Anunciante:", a)
    ok+=1


print(100*ok/len(tipo))
    

In [ ]:
primeira_linha = df.iloc[0]

# Convertendo colunas float para int

def float_to_int(value):
    return int(value) if not pd.isna(value) else int(0)

floats_ints = ['num_fotos','ano_modelo','hodometro']

for col in floats_ints:
    df[col] = df[col].apply(float_to_int)

# Percorrendo a primeira linha e imprimindo valor e tipo de cada variável
for coluna, valor in primeira_linha.iteritems():
    print(f'Coluna: {coluna}, Valor: {valor}, Tipo: {type(valor)}')

In [ ]:
# Convertendo as colunas boolean
# df['blindado']
def bool_blindado(value):
    if value=='S':
        return True
    if value=='N':
        return False
    
df['blindado'] = df['blindado'].apply(bool_blindado)

def bool_dono_aceita_troca(value):
    if isinstance(value, str) and value == 'Aceita troca':
        return True
    return False

df['dono_aceita_troca'] = df['dono_aceita_troca'].apply(bool_dono_aceita_troca)

def bool_veiculo_unico_dono(value):
    if isinstance(value, str) and value == 'Único dono':
        return True
    return False
    
df['veiculo_único_dono'] = df['veiculo_único_dono'].apply(bool_veiculo_unico_dono)

def bool_revisoes_concessionaria(value):
    if isinstance(value, str) and value == 'Todas as revisões feitas pela concessionária':
        return True
    return False

df['revisoes_concessionaria'] = df['revisoes_concessionaria'].apply(bool_revisoes_concessionaria)

def bool_ipva_pago(value):
    if isinstance(value, str) and value == 'IPVA pago':
        return True
    return False

df['ipva_pago'] = df['ipva_pago'].apply(bool_ipva_pago)

def bool_veiculo_licenciado(value):
    if isinstance(value, str) and value == 'Licenciado':
        return True
    return False

df['veiculo_licenciado'] = df['veiculo_licenciado'].apply(bool_veiculo_licenciado)

def bool_garantia_de_fabrica(value):
    if isinstance(value, str) and value == 'Garantia de fábrica':
        return True
    return False

df['garantia_de_fábrica'] = df['garantia_de_fábrica'].apply(bool_garantia_de_fabrica)

def bool_revisoes_dentro_agenda(value):
    if isinstance(value, str) and value == 'Todas as revisões feitas pela agenda do carro':
        return True
    return False

df['revisoes_dentro_agenda'] = df['revisoes_dentro_agenda'].apply(bool_revisoes_dentro_agenda)

df.info()
primeira_linha = df.iloc[0]
for coluna, valor in primeira_linha.iteritems():
    print(f'Coluna: {coluna}, Valor: {valor}, Tipo: {type(valor)}')


In [ ]:
for col in booleans:

    coluna = df[col].value_counts(dropna=False)
    ax = coluna.plot(kind="bar")
    plt.title('Value counts of '+col+' column')
    plt.xlabel(col)
    plt.xticks(rotation=0)
    plt.ylabel('Count')
    for i, v in enumerate(coluna):
        ax.text(i, v, str(v), ha='center', va='bottom', fontsize=12)
    plt.show()

In [ ]:
del df['versao']
del df['anunciante']
del df['elegivel_revisao']
del df['veiculo_alienado']

preco_coluna = df['preco']
del df['preco']
df['preco'] = preco_coluna

df

In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,1)
plt.title('Car Price Distribution Plot')
sns.distplot(df['preco'])
plt.gca().get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
# plt.gca().get_yaxis().set_major_formatter(plt.FuncFormatter(lambda y, loc: "{:,}".format(int(y))))

plt.subplot(1,2,2)
plt.title('Car Price Spread')
sns.boxplot(y=df['preco'])
plt.gca().get_yaxis().set_major_formatter(plt.FuncFormatter(lambda y, loc: "{:,}".format(int(y))))


plt.show()

In [ ]:
booleans = ['blindado','entrega_delivery','troca','dono_aceita_troca','veiculo_único_dono','revisoes_concessionaria','ipva_pago','veiculo_licenciado','garantia_de_fábrica','revisoes_dentro_agenda']
numericas = ['num_fotos','ano_de_fabricacao','ano_modelo','hodometro','num_portas']
strings = ['marca','modelo','cambio','tipo','cor','tipo_vendedor','cidade_vendedor','estado_vendedor','combustivel']

In [ ]:
def price_comp(a):
    fig = plt.figure(figsize=(10,3))
    s1=fig.add_subplot(1,2,1)
    s2=fig.add_subplot(1,2,2)


    count=df[a].value_counts()
    count.plot(kind='bar',ax=s1)
    s1.set_title(a+' Histogram')
    s1.set_xlabel(a)


    preco = pd.DataFrame(df.groupby([a])['preco'].mean().sort_values(ascending = False))
    preco.plot(kind='bar',ax=s2)
    plt.title(a+' vs Average Price')
    plt.show()
    
for col in strings:
    price_comp(col)

In [ ]:
    
for col in booleans:
    price_comp(col)

In [ ]:
def scatter(x,fig):
    plt.subplot(5,1,fig)
    plt.scatter(df[x],df['preco'])
    plt.title(x+' vs Preco')
    plt.ylabel('Preco')
    plt.xlabel(x)
    plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:,.0f}'.format(x)))

plt.figure(figsize=(10,20))

for i, col in enumerate(numericas):
    scatter(col,i+1)

plt.subplots_adjust(hspace=0.5)


In [ ]:
# Questões de negócio
#Foi considerado como popular carros abaixo de 120.000 reais.
carros_populares_abaixo_120000 = df[df['preco'] < 120000]


quantidade_por_estado = carros_populares_abaixo_120000['estado_vendedor'].value_counts().reset_index()
quantidade_por_estado.columns = ['estado_vendedor', 'quantidade']

quantidade_por_estado = quantidade_por_estado.sort_values(by='quantidade', ascending=False)

# Plotar o gráfico de barras com a quantidade de carros em cada estado
plt.figure(figsize=(12, 6))
plt.bar(quantidade_por_estado['estado_vendedor'], quantidade_por_estado['quantidade'])
plt.xlabel('Estado')
plt.ylabel('Quantidade de Carros')
plt.title('Quantidade de Carros com Preço Abaixo de R$ 120.000 por Estado')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Filtrar o DataFrame para obter apenas as picapes com transmissão automática
picapes_automaticas = df[(df['tipo'] == 'Picape') & (df['cambio'] == 'Automática')]

# Agrupar por estado e calcular a média dos preços para cada estado
media_preco_por_estado = picapes_automaticas.groupby('estado_vendedor')['preco'].mean().reset_index()

# Ordenar os estados pela média dos preços em ordem decrescente
media_preco_por_estado = media_preco_por_estado.sort_values(by='preco', ascending=True)

# Plotar o gráfico de barras
plt.figure(figsize=(12, 6))
plt.bar(media_preco_por_estado['estado_vendedor'], media_preco_por_estado['preco'])
plt.xlabel('Estado')
plt.ylabel('Média do Preço')
plt.title('Média do Preço das Picapes com Transmissão Automática por Estado')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
carros_garantia = df[df['garantia_de_fábrica'] == True]


media_preco_por_estado = carros_garantia.groupby('estado_vendedor')['preco'].mean().reset_index()

media_preco_por_estado = media_preco_por_estado.sort_values(by='preco', ascending=True)

# Obter o melhor estado
melhor_estado = media_preco_por_estado.iloc[0]['estado_vendedor']
melhor_media_preco = media_preco_por_estado.iloc[0]['preco']


plt.figure(figsize=(12, 6))
plt.bar(media_preco_por_estado['estado_vendedor'], media_preco_por_estado['preco'])
plt.xlabel('Estado')
plt.ylabel('Média do Preço')
plt.title('Média do Preço de Carros com Garantia de Fábrica por Estado')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Exibir o melhor estado e a respectiva métrica de desempenho
print(f'O melhor estado para se comprar carros com garantia de fábrica é {melhor_estado} com média de preço de R${melhor_media_preco:.2f}.')

In [ ]:
carros_garantia = df[df['garantia_de_fábrica'] == True]

# Calcular o IQR para a coluna 'preco'
Q1 = carros_garantia['preco'].quantile(0.25)
Q3 = carros_garantia['preco'].quantile(0.75)
IQR = Q3 - Q1


limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Remover os outliers com base no IQR
carros_garantia_sem_outliers = carros_garantia[(carros_garantia['preco'] >= limite_inferior) & (carros_garantia['preco'] <= limite_superior)]


media_preco_por_estado = carros_garantia_sem_outliers.groupby('estado_vendedor')['preco'].mean().reset_index()


media_preco_por_estado = media_preco_por_estado.sort_values(by='preco', ascending=False)


melhor_estado = media_preco_por_estado.iloc[0]['estado_vendedor']
melhor_media_preco = media_preco_por_estado.iloc[0]['preco']

plt.figure(figsize=(12, 6))
plt.bar(media_preco_por_estado['estado_vendedor'], media_preco_por_estado['preco'])
plt.xlabel('Estado')
plt.ylabel('Média do Preço')
plt.title('Média do Preço de Carros com Garantia de Fábrica por Estado')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



print(f'O melhor estado para se comprar carros com garantia de fábrica é {melhor_estado} com média de preço de R${melhor_media_preco:.2f}.')

In [ ]:

carros_eletricos_hibridos = df[df['combustivel'].isin(['ELETRICO', 'HIBRIDO'])]


quantidade_por_estado = carros_eletricos_hibridos['estado_vendedor'].value_counts().reset_index()
quantidade_por_estado.columns = ['estado_vendedor', 'quantidade']


quantidade_por_estado = quantidade_por_estado.sort_values(by='quantidade', ascending=False)


plt.figure(figsize=(12, 6))
plt.bar(quantidade_por_estado['estado_vendedor'], quantidade_por_estado['quantidade'])
plt.xlabel('Estado')
plt.ylabel('Quantidade de Carros Vendidos')
plt.title('Estados que mais venderam Carros Elétricos e Híbridos')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



In [ ]:
carros_populares = df[df['preco'] < 120000]

# Contar a quantidade de carros vendidos por marca
quantidade_por_marca = carros_populares['marca'].value_counts().reset_index()
quantidade_por_marca.columns = ['marca', 'quantidade']


quantidade_por_marca = quantidade_por_marca.sort_values(by='quantidade', ascending=False)

# Selecionar as 10 marcas mais vendidas
top_10_marcas = quantidade_por_marca.head(10)


plt.figure(figsize=(12, 6))
plt.bar(top_10_marcas['marca'], top_10_marcas['quantidade'])
plt.xlabel('Marca')
plt.ylabel('Quantidade de Carros Vendidos')
plt.title('Top 10 Marcas de Carros Populares Mais Vendidas')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:

# Filtrar o DataFrame para obter apenas carros de luxo (valor acima de 1 milhão)
carros_de_luxo = df[df['preco'] > 1000000]

# Contar a quantidade de carros de luxo vendidos por estado
vendas_por_estado = carros_de_luxo['estado_vendedor'].value_counts().reset_index()
vendas_por_estado.columns = ['estado_vendedor', 'quantidade']


estado_mais_vendas = vendas_por_estado.iloc[0]['estado_vendedor']
quantidade_vendas_estado_mais_vendas = vendas_por_estado.iloc[0]['quantidade']


plt.figure(figsize=(12, 6))
plt.bar(vendas_por_estado['estado_vendedor'], vendas_por_estado['quantidade'])
plt.xlabel('Estado')
plt.ylabel('Quantidade de Carros de Luxo Vendidos')
plt.title('Estados com Mais Vendas de Carros de Luxo (Acima de 1 Milhão)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
#Preparand os dados para criar modelo de classificação
booleans = ['blindado','garantia_de_fábrica']
numericas = ['ano_de_fabricacao','hodometro','preco']
strings = ['cambio','tipo','combustivel','modelo','marca']



df_filtrado = df.filter(booleans + numericas + strings)
for col in booleans:
    df_filtrado[col] = df_filtrado[col].astype(int)

new_df = pd.get_dummies(columns=strings,data=df_filtrado)
new_df

In [ ]:
x = new_df.drop(columns=["preco"])
y = new_df["preco"]

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
training_score = []
testing_score = []

In [ ]:
def model_prediction(model):
    model.fit(x_train,y_train)
    x_train_pred = model.predict(x_train)
    x_test_pred = model.predict(x_test)
    a = r2_score(y_train, x_train_pred) * 100
    b = r2_score(y_test, x_test_pred) * 100

    training_score.append(a)
    testing_score.append(b)


In [ ]:
model_prediction(LinearRegression())

In [ ]:
model_prediction(DecisionTreeRegressor())

In [ ]:
model_prediction(RandomForestRegressor())

In [ ]:
model_prediction(AdaBoostRegressor())

In [ ]:
model_prediction(GradientBoostingRegressor())

In [ ]:
model_prediction(LGBMRegressor())

In [ ]:
model_prediction(XGBRegressor())

In [ ]:
model_prediction(CatBoostRegressor(verbose=False))

In [ ]:
models = ["Linear Regression","Decision Tree","Random Forest","Ada Boost","Gradient Boost","LGBM","XGBoost","CatBoost"]
training_score = list(map(int, training_score))
testing_score = list(map(int, testing_score))
df = pd.DataFrame({"Algorithms":models,
                   "Training Score":training_score,
                   "Testing Score":testing_score})
df

In [ ]:
model = LGBMRegressor()

# Treinamento do modelo escolhido usando todos os dados de treinamento
model.fit(x, y)

In [ ]:
#carregando csv de teste para previsões e limpando os dados
df_teste = pd.read_csv('cars_test.csv', sep='\t', header=0, encoding="utf-16")

versao_teste = df_teste['versao']

df_teste['combustivel'] = None

for i,ver in enumerate(versao_teste):
    if "FLEX" in ver:
        df_teste.at[i, 'combustivel'] = 'FLEX'
        continue
    if "GASOLINA" in ver:
        df_teste.at[i, 'combustivel'] = 'GASOLINA'
        continue
    if "DIESEL" in ver:
        df_teste.at[i, 'combustivel'] = 'DIESEL'
        continue
    if "HÍBRIDO" in ver or "HYBRID" in ver:
        df_teste.at[i, 'combustivel'] = 'HIBRIDO'
        continue
    if "ELÉTRICO" in ver:
        df_teste.at[i, 'combustivel'] = 'ELETRICO'
        continue
    df_teste.at[i, 'combustivel'] = 'OUTROS'

    
df_teste_filtrado = df_teste.filter(booleans + numericas + strings)
df_teste_filtrado['hodometro'] = df_teste_filtrado['hodometro'].apply(float_to_int)
df_teste_filtrado['blindado'] = df_teste_filtrado['blindado'].apply(bool_blindado)
df_teste_filtrado['garantia_de_fábrica'] = df_teste_filtrado['garantia_de_fábrica'].apply(bool_garantia_de_fabrica)
for col in booleans:
    df_teste_filtrado[col] = df_teste_filtrado[col].astype(int)
new_df_teste = pd.get_dummies(columns=strings,data=df_teste_filtrado)
new_df_teste

In [ ]:
colunas_a_adicionar = x.columns.difference(new_df_teste.columns)
colunas_a_remover = new_df_teste.columns.difference(x.columns)

for col in colunas_a_adicionar:
    new_df_teste[col] = 0
    
for col in colunas_a_remover:
    del new_df_teste[col]
    
pred = model.predict(new_df_teste)
pred

In [ ]:
df_final = pd.DataFrame({'id': df_teste['id'], 'preco': pred})

# Salvar o novo DataFrame em um arquivo CSV com separador de tabulação
df_final.to_csv('predicted.csv ', index=False, sep='\t', encoding='utf-16')